# Daily Average Glucose

This notebook generates daily average glucose chart from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service:

- A line plot for daily mean
- +/- 1 standard deviation shaded area, indicating daily variability
- Target zone indicator

Both charts include the median line.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, datetime, timedelta

# Reporting date range
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_chart_export_format = "PNG"

In [ ]:
# Set the percentile limits
lower_percentile = 10
upper_percentile = 90

In [ ]:
%run api.ipynb
%run config.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the blood pressure readings
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
# Rename the date column to timestamp and generate a date-only column
df.rename(columns={ "date": "timestamp" }, inplace=True)
df['date'] = df['timestamp'].dt.date

# Group by date and calculate mean and standard deviation
daily_stats = df.groupby('date')['level'].agg(['mean', 'std']).reset_index()

# Preview the data
daily_stats.head()

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Define the export file name
export_file_name = "Glucose-Daily-Mean"
output_path = export_folder_path / f"{export_file_name}.xlsx"

# Export the data to a spreadsheet
if export_spreadsheet:
    with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
        daily_stats.to_excel(writer, sheet_name="Daily Mean", index=False)

In [ ]:
import matplotlib.pyplot as plt

# Plot setup
plt.figure(figsize=(12, 6))

# Plot daily mean glucose
plt.plot(daily_stats['date'], daily_stats['mean'], label='Daily Mean Glucose (mmol/L)', marker='o')

# Plot ±1 standard deviation as a shaded area
plt.fill_between(
    daily_stats['date'],
    daily_stats['mean'] - daily_stats['std'],
    daily_stats['mean'] + daily_stats['std'],
    color='blue',
    alpha=0.2,
    label='±1 SD'
)

# Add target range band: 3.9–10.0 mmol/L
plt.axhspan(3.9, 10.0, color='green', alpha=0.1, label='Target Range (3.9–10.0 mmol/L)')

# Formatting
plt.title('Daily Average Glucose with Variability and Target Range')
plt.xlabel('Date')
plt.ylabel('Glucose (mmol/L)')
plt.xticks(rotation=45)
plt.grid(True)

# Legend below chart
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=2)
plt.tight_layout()

# Export to PNG
if chart_chart_export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if chart_chart_export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()
